In [1]:
import torch
from utils.utils import *
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils import data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import warnings
from torch.utils.data.dataset import Dataset
from PIL import Image

Our targets
   > If first number is larger than second number = 0 
   
   > If the numbers are equal = 1
   
   > If the second number is larger than the first number = 1

In [2]:
class MnistPairs(Dataset):
    
    training_file = 'training.pt'
    test_file = 'test.pt'

    @property
    def train_labels(self):
        warnings.warn("train_labels has been renamed targets")
        return self.targets

    @property
    def test_labels(self):
        warnings.warn("test_labels has been renamed targets")
        return self.targets

    @property
    def train_data(self):
        warnings.warn("train_data has been renamed data")
        return self.data

    @property
    def test_data(self):
        warnings.warn("test_data has been renamed data")
        return self.data
    
    @property
    def train_classes(self):
        warnings.warn("train_classes has been renamed classes")
        return self.classes
    
    @property
    def test_classes(self):
        warnings.warn("test_classes has been renamed classes")
        return self.classes
    
    @property
    def processed_folder(self):
        return os.path.join(self.root, self.__class__.__name__, 'processed')
    
    def _check_exists(self):
        return os.path.exists(os.path.join(self.processed_folder, self.training_file)) and \
            os.path.exists(os.path.join(self.processed_folder, self.test_file))
    
    def mnist_to_pairs(self, nb, input, target):
        input = torch.functional.F.avg_pool2d(input, kernel_size = 2)
        a = torch.randperm(input.size(0))
        a = a[:2 * nb].view(nb, 2)
        input = torch.cat((input[a[:, 0]], input[a[:, 1]]), 1)
        classes = target[a]
        target = (classes[:, 0] <= classes[:, 1]).long()
        return input, target, classes

    def generate_pair_sets(self, nb):
        
        if self.train:
            dataset = datasets.MNIST(self.root + '/mnist/', train = True, download = True)
            dataset_input = dataset.train_data.view(-1, 1, 28, 28).float()
            dataset_target = dataset.train_labels
        
        else:
            dataset = datasets.MNIST(self.root + '/mnist/', train = False, download = True)
            dataset_input = dataset.test_data.view(-1, 1, 28, 28).float()
            dataset_target = dataset.test_labels

        return self.mnist_to_pairs(nb, dataset_input, dataset_target)
    
    def __init__(self, root, nb = 1000, train=True, transform=None, target_transform=None):

        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set
        
        if not self._check_exists():
            raise RuntimeError('Dataset not found.')
            
        if self.train:
            data_file = self.training_file
        else:
            data_file = self.test_file
        
        self.data ,self.targets, self.classes = self.generate_pair_sets(nb)
        
    
    
    def __getitem__(self, index):
        
        img, target = self.data[index], int(self.targets[index])
        
        img_1 = Image.fromarray(img[0].numpy(), mode='L')
        img_2 = Image.fromarray(img[1].numpy(), mode='L')
        
        if self.transform is not None:
            img_1 = self.transform(img_1)
            img_2 = self.transform(img_2)
            
            img = torch.stack((img_1.reshape(img_1.shape[1],img_1.shape[2]),img_2.reshape(img_2.shape[1],img_2.shape[2])),dim=0)
        if self.target_transform is not None:
            target = self.target_transform(target)
            
        return img, target

    def __len__(self):
        return len(self.data)

In [3]:
#Loading dataset
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

train_dataset = MnistPairs('data/',train=True, transform=None)
test_dataset = MnistPairs('data/',train=False, transform=None)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=128, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=128, 
                                          shuffle=False)

In [61]:
#Defining the network          
class CNNModel(nn.Module):
    
    def __init__(self):
        super(CNNModel, self).__init__()
        
        #Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=2, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        
        #Max pool 1
        #self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        #Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        #Max pool 2
        #self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        #Dropout for regularization
        self.dropout = nn.Dropout(p=0.2)
        
        #Fully Connected 1
        self.fc1 = nn.Linear(32*14*14, 2)
        
    def forward(self, x):
        #Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        #Max pool 1
        #out = self.maxpool1(out)
        
        #Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        
        #Max pool 2
        #out = self.maxpool2(out)
        
        #Resize
        out = out.view(out.size(0), -1)
        
        #Dropout
        out = self.dropout(out)
        
        #Fully connected 1
        out = self.fc1(out)
        return out

In [62]:
#Create instance of model
model = CNNModel()
#Create instance of loss
criterion = nn.CrossEntropyLoss()
#Create instance of optimizer (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [63]:
#Train the model
iter = 0
for epoch in range(25):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        #Clear the gradients
        optimizer.zero_grad()
        
        #Forward propagation 
        outputs = model(images)      
        
        #Calculating loss with softmax to obtain cross entropy loss
        loss = criterion(outputs, labels)
        
        #Backward propation
        loss.backward()
        
        #Updating gradients
        optimizer.step()
        
        iter += 1
        
        #Total number of labels
        total = labels.size(0)
        
        #Obtaining predictions from max value
        _, predicted = torch.max(outputs.data, 1)
        
        #Calculate the number of correct answers
        correct = (predicted == labels).sum().item()
        
        #Print loss and accuracy
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
              .format(epoch + 1, 25, i + 1, len(train_loader), loss.item(),
                      (correct / total) * 100))

Epoch [1/25], Step [1/8], Loss: 6.4525, Accuracy: 46.09%
Epoch [1/25], Step [2/8], Loss: 43.5459, Accuracy: 51.56%
Epoch [1/25], Step [3/8], Loss: 18.1544, Accuracy: 53.91%
Epoch [1/25], Step [4/8], Loss: 4.9901, Accuracy: 53.12%
Epoch [1/25], Step [5/8], Loss: 1.1381, Accuracy: 53.12%
Epoch [1/25], Step [6/8], Loss: 0.9469, Accuracy: 58.59%
Epoch [1/25], Step [7/8], Loss: 1.0300, Accuracy: 47.66%
Epoch [1/25], Step [8/8], Loss: 1.1054, Accuracy: 40.38%
Epoch [2/25], Step [1/8], Loss: 0.8472, Accuracy: 47.66%
Epoch [2/25], Step [2/8], Loss: 0.7792, Accuracy: 50.00%
Epoch [2/25], Step [3/8], Loss: 0.7520, Accuracy: 47.66%
Epoch [2/25], Step [4/8], Loss: 0.7228, Accuracy: 50.78%
Epoch [2/25], Step [5/8], Loss: 0.6695, Accuracy: 58.59%
Epoch [2/25], Step [6/8], Loss: 0.6613, Accuracy: 57.03%
Epoch [2/25], Step [7/8], Loss: 0.6831, Accuracy: 53.12%
Epoch [2/25], Step [8/8], Loss: 0.6684, Accuracy: 59.62%
Epoch [3/25], Step [1/8], Loss: 0.6668, Accuracy: 59.38%
Epoch [3/25], Step [2/8], Los

Epoch [19/25], Step [2/8], Loss: 0.3166, Accuracy: 85.94%
Epoch [19/25], Step [3/8], Loss: 0.3485, Accuracy: 83.59%
Epoch [19/25], Step [4/8], Loss: 0.2943, Accuracy: 87.50%
Epoch [19/25], Step [5/8], Loss: 0.3280, Accuracy: 84.38%
Epoch [19/25], Step [6/8], Loss: 0.3302, Accuracy: 82.81%
Epoch [19/25], Step [7/8], Loss: 0.3803, Accuracy: 79.69%
Epoch [19/25], Step [8/8], Loss: 0.2770, Accuracy: 88.46%
Epoch [20/25], Step [1/8], Loss: 0.3715, Accuracy: 82.81%
Epoch [20/25], Step [2/8], Loss: 0.3379, Accuracy: 83.59%
Epoch [20/25], Step [3/8], Loss: 0.3492, Accuracy: 88.28%
Epoch [20/25], Step [4/8], Loss: 0.2988, Accuracy: 85.94%
Epoch [20/25], Step [5/8], Loss: 0.3362, Accuracy: 87.50%
Epoch [20/25], Step [6/8], Loss: 0.3129, Accuracy: 87.50%
Epoch [20/25], Step [7/8], Loss: 0.3810, Accuracy: 82.03%
Epoch [20/25], Step [8/8], Loss: 0.2652, Accuracy: 89.42%
Epoch [21/25], Step [1/8], Loss: 0.3769, Accuracy: 85.16%
Epoch [21/25], Step [2/8], Loss: 0.3277, Accuracy: 88.28%
Epoch [21/25],

In [85]:
#Testing the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = Variable(images)
        labels = Variable(labels)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 1000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 1000 test images: 71.7 %
